In [35]:
#load_ext autotime
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook

In [2]:
locator = Nominatim(user_agent="measurements")

In [3]:
odf = pd.read_csv(r"T:\MPO\Bike&Ped\BikeCounting\StoryMap\BikeShare\Data\bike_share_locations.csv")
odf.head()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination
0,14243380,19570,2210370,NaN,44.04497666666666,-123.08334333333333,2022-03-01,00:07,Origin
1,14243383,19612,2208820,17th & Moss,44.04110166666667,-123.06669,2022-03-01,00:55,Origin
2,14243387,19606,2199102,NaN,44.03341833333333,-123.0724,2022-03-01,02:39,Origin
3,14243431,19753,1366158,Matthew Knight Arena,44.045363333333334,-123.065185,2022-03-01,05:38,Origin
4,14243442,19797,2221341,PeaceHealth University District,44.047685,-123.081865,2022-03-01,05:56,Origin


In [4]:
'Knight Library (Virtual Hub)' in odf.Location.unique()

True

In [5]:
odf[odf.Latitude.isnull() | odf.Longitude.isnull()]

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination


In [6]:
" - " in odf.Latitude.values

True

In [7]:
df = odf

In [8]:
df = df[df.Latitude != " - "]

In [9]:
" - " in df.Latitude.values

False

In [10]:
def silence_pandas(silence):
    if silence:
        pd.options.mode.chained_assignment = None
    else:
        pd.options.mode.chained_assignment = 'warn'

In [11]:
silence_pandas(True)

In [12]:
df["geom"] = df["Latitude"].map(str) + ',' + df['Longitude'].map(str)
df["geom"][0]

'44.04497666666666,-123.08334333333333'

In [13]:
locator = Nominatim(user_agent="measurements", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)

In [14]:
tqdm_notebook.pandas()

In [15]:
df['address'] = df['geom'].progress_apply(rgeocode)
df.head()

  0%|          | 0/19356 [00:00<?, ?it/s]

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination,geom,address
0,14243380,19570,2210370,NaN,44.04497666666666,-123.08334333333333,2022-03-01,00:07,Origin,"44.04497666666666,-123.08334333333333","(611, East 13th Alley, Eugene, Lane County, Or..."
1,14243383,19612,2208820,17th & Moss,44.04110166666667,-123.06669,2022-03-01,00:55,Origin,"44.04110166666667,-123.06669","(Kalapuya Illhi, East 17th Avenue, Eugene, Lan..."
2,14243387,19606,2199102,NaN,44.03341833333333,-123.0724,2022-03-01,02:39,Origin,"44.03341833333333,-123.0724","(1352, East 24th Avenue, College Hill, Eugene,..."
3,14243431,19753,1366158,Matthew Knight Arena,44.045363333333334,-123.065185,2022-03-01,05:38,Origin,"44.045363333333334,-123.065185","(Matthew Knight Arena, Franklin Boulevard, Eug..."
4,14243442,19797,2221341,PeaceHealth University District,44.047685,-123.081865,2022-03-01,05:56,Origin,"44.047685,-123.081865","(Emerald Express, Eugene, Lane County, Oregon,..."


In [16]:
df.address[1]

Location(Kalapuya Illhi, East 17th Avenue, Eugene, Lane County, Oregon, 97403, United States, (44.041454900000005, -123.0672510098804, 0.0))

In [17]:
df.Location.isnull().values[3]

False

In [18]:
def getLoc(x):
    loc = x.raw['address']
    a = [key for key, value in loc.items()][0]
    b = [key for key, value in loc.items()][1]
    if  a == 'house_number':
        res = loc['road'] + ' ' + loc['house_number']
    else:
        res = loc[a]
        if res == "University of Oregon":
            res = "UO " + loc[b]
    return res

In [19]:
getLoc(df[df.Location.isnull().values].address.values[1])

'East 24th Avenue 1352'

In [20]:
df['Name'] = df.address.apply(lambda x: getLoc(x))

In [21]:
sorted(df.Name.unique())

['13th & Olive',
 '13th Avenue Market',
 '13th and Olive (Willamate)',
 '16 Tons Café',
 '16 Tons Taphouse & Bottleshop',
 '2125 Franklin',
 '4 Plus 3 Korean Barbaque',
 '5000s - Brooks Field',
 '5000s/6000s Brooks Field',
 '5th Street Public Market',
 '66 Motel',
 '6th Street Alley',
 '7-Eleven',
 '8000s - Brooks Field',
 '959 Franklin Apartments',
 'Adams Street 350',
 'Adams Street 366',
 'Agate Alley Bistro',
 'Agate Street 1840',
 'Alder Alley 1705',
 'Alder Alley 2200',
 'Alder Street 1848',
 'Alder Street 1950',
 'Alder Street 2388',
 'Alder Street 950',
 "All Thai'd Up",
 'Allen Price Science Library',
 'Alpha Chi Omega',
 'Alton Baker Disc Golf Course',
 'Alton Baker Dog Park',
 'Amazon Locker',
 'Amazon Parkway',
 'Amazon Parkway 2982',
 'Amazon Pool',
 'Amazon Pool Facility',
 'Ambrosia Restaurant & Bar',
 'Anderson Lane 811',
 "Andrea's Red Dress",
 'Andy',
 'Anstett Hall',
 'Approaching UO Station',
 'Autzen Stadium',
 "BJ's",
 'Bare Bones Dog Wash',
 'Bari Trattoria White

In [22]:
for n in df.loc[df.Location.isnull(),'Name'].unique():
    if all(df.loc[df['Name'] == n, 'Location'].isnull()):
        df.loc[df['Name'] == n, 'Location'] = n
    else:
        v = df.loc[df['Name'] == n, 'Location']
        df.loc[((df.Location.isnull()) & df['Name'] == n), 'Location'] = v[v.notnull()].values[0]    

In [23]:
df['Location'] = df['Location'].str.replace('University of Oregon','UO')

In [24]:
def remove_street_number(x):
    x = str(x)
    if len(x.split(' ')[-2:-1]) == 0:
        res = x
    elif (x.split(' ')[-2:-1][0] in ['Avenue', 'Street', 'Alley']) and (x.split(' ')[-1:][0].isdigit()):
        res = x.rsplit(' ', 1)[0]
    else:
        res = x
    
    return res

In [25]:
df['Location'] = df['Location'].apply(lambda x: remove_street_number(x))

In [26]:
df['Location'] = df['Location'].str.replace(' -',',')

In [27]:
df.loc[df.Location == 'nan', 'Name']

64                                         UO Sunset Alley
113                                            Wells Fargo
119                                        UO Sunset Alley
157                             Boone Insurance Associates
158                             Boone Insurance Associates
                               ...                        
19254    University of Oregon Library and Memorial Quad...
19263                                      UO Sunset Alley
19287                                      UO Sunset Alley
19302                                     Market of Choice
19324                                      UO Sunset Alley
Name: Name, Length: 1128, dtype: object

In [28]:
df.loc[df.Location == 'nan', 'Location'] = df.loc[df.Location == 'nan', 'Name']

In [29]:
for i in df.RouteID.unique():
    idx1 = ((df['RouteID'] == i) & (df['OriginDestination'] =='Origin'))
    idx2 = ((df['RouteID'] == i) & (df['OriginDestination'] =='Destination'))
    pathID = df.loc[idx1, 'Location'].values[0] + ' - ' + df.loc[idx2, 'Location'].values[0]
    df.loc[idx1, 'PathID'] = pathID
    df.loc[idx2, 'PathID'] = pathID

In [30]:
df.tail()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination,geom,address,Name,PathID
19359,14268705,19604,2167848,13th & Kincaid,44.04589333333333,-123.078505,2022-03-31,23:12,Destination,"44.04589333333333,-123.078505","(Chiles Hall, University Station North - Bay C...",Chiles Hall,19th and Agate - 13th & Kincaid
19360,14268715,19683,2200344,PeaceHealth University District,44.047615,-123.08192333333334,2022-03-31,23:20,Destination,"44.047615,-123.08192333333334",(Peace Health Medical Center University Distri...,Peace Health Medical Center University District,13th & Kincaid - PeaceHealth University District
19361,14268717,19749,1702550,Franklin Blvd and Walnut North,44.045253333333335,-123.06140666666667,2022-03-31,23:27,Destination,"44.045253333333335,-123.06140666666667","(1237, Walnut Street, Eugene, Lane County, Ore...",Walnut Street 1237,13th & Agate (Virtual Hub) - Franklin Blvd and...
19362,14268723,19788,2217628,The 515,44.050048333333336,-123.08449833333333,2022-03-31,23:55,Destination,"44.050048333333336,-123.08449833333333","(The 515, 515, East Broadway, Eugene, Lane Cou...",The 515,UO Sunset Alley - The 515
19363,14268725,19675,2207782,13th & Agate (Virtual Hub),44.04534,-123.06954666666667,2022-04-01,00:12,Destination,"44.04534,-123.06954666666667","(University of Oregon, Sunset Alley, Eugene, L...",UO Sunset Alley,10th & Olive NE - 13th & Agate (Virtual Hub)


In [31]:
ndf = df.drop(['geom', 'address'], axis=1)

In [32]:
ndf.tail()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination,Name,PathID
19359,14268705,19604,2167848,13th & Kincaid,44.04589333333333,-123.078505,2022-03-31,23:12,Destination,Chiles Hall,19th and Agate - 13th & Kincaid
19360,14268715,19683,2200344,PeaceHealth University District,44.047615,-123.08192333333334,2022-03-31,23:20,Destination,Peace Health Medical Center University District,13th & Kincaid - PeaceHealth University District
19361,14268717,19749,1702550,Franklin Blvd and Walnut North,44.045253333333335,-123.06140666666667,2022-03-31,23:27,Destination,Walnut Street 1237,13th & Agate (Virtual Hub) - Franklin Blvd and...
19362,14268723,19788,2217628,The 515,44.050048333333336,-123.08449833333333,2022-03-31,23:55,Destination,The 515,UO Sunset Alley - The 515
19363,14268725,19675,2207782,13th & Agate (Virtual Hub),44.04534,-123.06954666666667,2022-04-01,00:12,Destination,UO Sunset Alley,10th & Olive NE - 13th & Agate (Virtual Hub)


In [33]:
ndf['Name'] = ndf['Name'].str.replace('University of Oregon','UO')
ndf['Name'] = ndf['Name'].str.replace('U of O','UO')

In [34]:
ndf.to_csv(r'T:\MPO\Bike&Ped\BikeCounting\StoryMap\BikeShare\Data\pathID.csv', index=False)